In [1]:
import torch
import torchvision
from torch.autograd import Variable
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [2]:
!nvidia-smi

Tue Jul  9 16:43:52 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.12                 Driver Version: 390.12                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:3D:00.0 Off |                    0 |
| N/A   71C    P0    56W / 149W |    827MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:3E:00.0 Off |                    0 |
| N/A   63C    P0   109W / 149W |   7406MiB / 11441MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.5],std=[0.5])])
data_train = datasets.MNIST(root='./data',train=True,transform=transform)
data_test = datasets.MNIST(root='./data',train=False,transform=transform)
data_train_loader = torch.utils.data.DataLoader(dataset=data_train,batch_size=64,shuffle=True)
data_test_loader = torch.utils.data.DataLoader(dataset=data_test,batch_size=64,shuffle=True)

In [4]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.RNN(
            input_size = 28,
            hidden_size = 128,
            num_layers = 1,
            batch_first = True
        )
        self.fc = torch.nn.Linear(128,10)
    def forward(self,x):
        x,_ = self.rnn(x)
        x = self.fc(x[:,-1,:])
        return x

In [5]:
model = Model().cuda()
epochs = 10
optimizer = torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch,epochs))
    print("Training")
    print("-"*20)
    
    los = 0.0
    correct = 0.0
    start = time.time()
    for data in data_train_loader:
        X_train,y_train = data
        X_train = X_train.view(-1,28,28)
        X_train,y_train = Variable(X_train),Variable(y_train)
        X_train,y_train = X_train.cuda(),y_train.cuda()
        optimizer.zero_grad()
        outputs = model(X_train)
        _,pred = torch.max(outputs,1)
        correct += torch.sum(pred == y_train)
        losses = loss(outputs,y_train)
        losses.backward()
        los += losses
        optimizer.step()
    
    length = len(data_train)
    print("Time used:{:.4f}s".format(time.time()-start))
    print("Training loss:{:.4f},accuracy:{:.4f}%".format(los/length,100*correct.float()/length))
    
    print("Testing")
    print("-"*20)
    
    los = 0.0
    correct = 0.0
    start = time.time()
    for data in data_test_loader:
        X_train,y_train = data
        X_train = X_train.view(-1,28,28)
        X_train,y_train = Variable(X_train),Variable(y_train)
        X_train,y_train = X_train.cuda(),y_train.cuda()
        outputs = model(X_train)
        _,pred = torch.max(outputs,1)
        correct += torch.sum(pred == y_train)
        losses = loss(outputs,y_train)
        los += losses
    
    length = len(data_test)
    print("Time used:{:.4f}s".format(time.time()-start))
    print("Testing loss:{:.4f},accuracy:{:.4f}%".format(los/length,100*correct.float()/length))

Epoch 0/10
Training
--------------------
Time used:16.9207s
Training loss:0.0118,accuracy:75.0467%
Testing
--------------------
Time used:2.2728s
Testing loss:0.0061,accuracy:88.1100%
Epoch 1/10
Training
--------------------
Time used:17.2781s
Training loss:0.0048,accuracy:90.8233%
Testing
--------------------
Time used:2.1518s
Testing loss:0.0037,accuracy:93.3900%
Epoch 2/10
Training
--------------------
Time used:17.3111s
Training loss:0.0037,accuracy:93.1333%
Testing
--------------------
Time used:2.1519s
Testing loss:0.0037,accuracy:92.7900%
Epoch 3/10
Training
--------------------
Time used:17.3092s
Training loss:0.0031,accuracy:94.2800%
Testing
--------------------
Time used:2.1638s
Testing loss:0.0031,accuracy:94.5000%
Epoch 4/10
Training
--------------------
Time used:17.4839s
Training loss:0.0028,accuracy:94.8983%
Testing
--------------------
Time used:2.1657s
Testing loss:0.0028,accuracy:94.9000%
Epoch 5/10
Training
--------------------
Time used:17.4789s
Training loss:0.0025